In [47]:
from langchain.document_loaders.csv_loader import CSVLoader
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS 
from langchain_openai import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory 

In [28]:
import os
#replace it with your own key
os.environ["OPENAI_API_KEY"] = 'sk-proj-...'

In [29]:
loader = CSVLoader(file_path="faq.csv")
documents = loader.load()
print(documents)

[Document(metadata={'source': 'faq.csv', 'row': 0}, page_content='Question: What are your pricing plans?\nAnswer: What are your pricing plans?\nMonthly plan $29/month\nYearly plan $129/year\nLifetime Membership $399\nTeam Yearly plan up to 7 team members $299/year\nTeam Lifetime Membership up to 7 team members $799'), Document(metadata={'source': 'faq.csv', 'row': 1}, page_content='Question: What am I getting as a Premium Member?\nAnswer: You will get access to:\n- 40+ video and text-form courses\n- Github repositories for those courses\n- 50+ in-depth tutorials marked as "premium"\n- Access to private Discord to ask questions'), Document(metadata={'source': 'faq.csv', 'row': 2}, page_content="Question: Do I get charged automatically for upcoming months/years?\nAnswer: We use Paddle as our payment merchant, which automatically charges your account every month/year (depending on your plan), until you manually decide to cancel.\n\nIt's a typical SaaS behavior, when you subscribe to the o

In [31]:
embeddings_model = OpenAIEmbeddings() 

In [15]:
# Just an example from the official docs:
embeddings = embeddings_model.embed_documents(
    [
        "Hi there!",
        "Oh, hello!"
    ]
)
print(embeddings)

[[-0.020291371271014214, -0.007072774693369865, -0.022869061678647995, -0.02623402513563633, -0.03748443350195885, 0.021688148379325867, -0.006199788302183151, -0.009021914564073086, 0.008463202975690365, -0.01654546707868576, 0.02684352919459343, -0.007447364740073681, -0.01358683779835701, -0.024176953360438347, 0.0064854929223656654, -0.0202659759670496, 0.024278536438941956, -0.01469156239181757, 0.016418486833572388, -0.016558164730668068, -0.007256895303726196, -0.008120357990264893, 0.0046347626484930515, -0.0020491364412009716, -0.014843937940895557, -0.006047412753105164, -0.0020697705913335085, -0.022957947105169296, 0.019897732883691788, -0.031617969274520874, 0.012888449244201183, 0.01170753687620163, -0.008545740507543087, -0.009548880159854889, -0.0018237472977489233, -0.027402238920331, -0.008253686130046844, 0.0021189753897488117, 0.023986482992768288, -0.008767955005168915, 0.023542053997516632, 0.0009079055162146688, 0.009783793240785599, -0.013942381367087364, -0.017

In [32]:
# storing vectors
vectorstore = FAISS.from_documents(documents, embeddings_model) 
# Return 3 most relevant documents
vectorstore.as_retriever(search_kwargs={"k": 3})

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001B649678BF0>, search_kwargs={'k': 3})

In [33]:
#test similarity search
results = vectorstore.similarity_search('Do you have a free plan?')
print(results)

[Document(metadata={'source': 'faq.csv', 'row': 0}, page_content='Question: What are your pricing plans?\nAnswer: What are your pricing plans?\nMonthly plan $29/month\nYearly plan $129/year\nLifetime Membership $399\nTeam Yearly plan up to 7 team members $299/year\nTeam Lifetime Membership up to 7 team members $799'), Document(metadata={'source': 'faq.csv', 'row': 2}, page_content="Question: Do I get charged automatically for upcoming months/years?\nAnswer: We use Paddle as our payment merchant, which automatically charges your account every month/year (depending on your plan), until you manually decide to cancel.\n\nIt's a typical SaaS behavior, when you subscribe to the online services."), Document(metadata={'source': 'faq.csv', 'row': 5}, page_content='Question: Any discount coupons based on country? PPP?\nAnswer: Yes. All Purchasing Power Parity discounts for our LaravelDaily products are available here: https://coupons.laraveldaily.com'), Document(metadata={'source': 'faq.csv', 'r

In [48]:
#Build a Conversation Chain via LangChain
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True) 
llm_model = ChatOpenAI()
chain = ConversationalRetrievalChain.from_llm(llm=llm_model, 
                                              retriever=vectorstore.as_retriever(),
                                             memory=memory) 

In [49]:
#Finally: Ask The Chatbot
query = "Do you have a team plan?" 
response = chain.invoke({"question": query}) 
 
query2 = 'And what about free plan?' 
response = chain.invoke({"question": query2}) 
 
print(response['chat_history']) 

[HumanMessage(content='Do you have a team plan?'), AIMessage(content='Yes, we do have team plans available. The Team Yearly plan allows up to 7 team members for $299/year, and the Team Lifetime Membership for up to 7 team members is priced at $799.'), HumanMessage(content='And what about free plan?'), AIMessage(content="I don't have information on the free plan.")]
